In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

data = pd.DataFrame({
    'title': ['Chinese Beijing Chinese', 'Chinese Chinese Shanghai', 'Chinese Macao', 'Tokyo Japan Chinese'],
    'category': ['c', 'c', 'c', 'j']
})

vocab = list(set(' '.join(data['title']).split()))
print("Unique words: ")
print(vocab)

def title_to_bow(title):
    bow = np.zeros(len(vocab))
    for i, word in enumerate(vocab):
        bow[i] = title.count(word)
    return bow

data['bow'] = data['title'].apply(title_to_bow)

test_sentence = input('Enter a test sentence: ')



In [ ]:
test_bow = title_to_bow(test_sentence)

class_counts = data['category'].value_counts()
prior_probs = class_counts / len(data)

flag = True
for a in test_sentence.split(" "):
    if vocab.count(a) > 0:
        flag = False
        break

if flag :
    print("NOTE: The is Test sentence is not within the scope of Vocabulary")

word_counts = pd.DataFrame(np.zeros((len(vocab), 2)), index=vocab, columns=['c', 'j'])
for i, row in data.iterrows():
    category = row['category']
    for j, count in enumerate(row['bow']):
        word_counts.loc[vocab[j], category] += count

cond_probs = (word_counts + 1) / (word_counts.sum() + len(vocab))

probs = np.ones(2)
for j, count in enumerate(test_bow):
    word = vocab[j]
    print('P({}| c) = ({:.0f} + 1) / ({:.0f} + {}) = {:.6f}'.format(word, word_counts.loc[word, 'c'], word_counts['c'].sum(), len(vocab), cond_probs.loc[word, 'c']))
    print('P({} | j) = ({:.0f} + 1) / ({:.0f} + {}) = {:.6f}'.format(word, word_counts.loc[word, 'j'], word_counts['j'].sum(), len(vocab), cond_probs.loc[word, 'j']))
    probs = cond_probs.iloc[j] * count
probs *= prior_probs 
if probs[0] > probs[1]:
    prediction = 'c'
else:
    prediction = 'j'

probs = np.ones(2)
print('Calculations for Classification of Test Sentence:\n')

print('P(c) =', prior_probs['c'])
print('P(j) =', prior_probs['j'])
print()
for j, count in enumerate(test_bow):
    probs = cond_probs.iloc[j] * count
probs *= prior_probs
print('P(c|test_sentence) =', probs[0])
print('P(j|test_sentence) =', probs[1])
if probs[0] > probs[1]:
    prediction = 'c'
else:
    prediction = 'j'

print('Test sentence:', test_sentence)
print('Prediction:', prediction)